<a href="https://colab.research.google.com/github/villasen/colab_test_notebook/blob/master/wav_to_quantized_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import hashlib
import math
import os.path
import random
import re
import sys
import tarfile
import argparse

from google.colab import files

import numpy as np
from six.moves import urllib
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf

from tensorflow.contrib.framework.python.ops import audio_ops as contrib_audio
from tensorflow.python.ops import io_ops
from tensorflow.python.platform import gfile
from tensorflow.python.util import compat
from tensorflow.python.ops import array_ops


FLAGS = None

output_c_file = "/content/wav_data.h"
#output_c_file = "/Users/martin/machine_learning/my_ML-KWS-for-MCU/ML-KWS-for-MCU/output_c_file2.h"
#output_c_file = "output_c_file.h"
my_input_wav = "/content/0ba018fc_nohash_2.wav"


def load_wav_file(my_input_wav):
    """Loads an audio file and returns a float PCM-encoded array of samples.

    Args:
      filename: Path to the .wav file to load.

    Returns:
      Numpy array holding the sample data as floats between -1.0 and 1.0.
    """
    variable_base = os.path.splitext(os.path.basename(my_input_wav).lower())[0]
    print(variable_base)

    with tf.Session(graph=tf.Graph()) as sess:
        wav_filename_placeholder = tf.placeholder(tf.string, [])
        wav_loader = io_ops.read_file(wav_filename_placeholder)
        wav_decoder = contrib_audio.decode_wav(wav_loader, desired_channels=1)
        return sess.run(
            wav_decoder,
            feed_dict={wav_filename_placeholder: my_input_wav}).audio.flatten()


def quantize_wav(wav_tensor):
    with tf.Session(graph=tf.Graph()) as sess:
        my_quant = tf.quantization.quantize(
            # input
            wav_tensor,
            #min_range    -32768
            -1.0,
            # max_range    32677
            1.0,
            # T
            tf.qint16,
            # mode='MIN_COMBINED',
            mode='SCALED',
            # round_mode='HALF_AWAY_FROM_ZERO'
            round_mode='HALF_AWAY_FROM_ZERO',
            name=None)
        return sess.run(my_quant).output

print("Starting")

# We want to see all the logging messages.
tf.logging.set_verbosity(tf.logging.INFO)
#wav_tensor = load_wav_file(FLAGS.my_input_wav,FLAGS.output_c_file)
wav_tensor = load_wav_file(my_input_wav)
print("wave data tensor")
print(wav_tensor)
print("Quantized data tensor")
quant_wav_tensor = quantize_wav(wav_tensor)
print(quant_wav_tensor)
size = len(quant_wav_tensor)
print(size)
#tf.logging.info('Wrote to "%s"' % (FLAGS.output_c_file))
""" for value in quant_wav_tensor:
    i=0
    value[i] = quant_wav_tensor[i]
    i=i+1
    return value
"""

with gfile.GFile(output_c_file, 'w') as f:
    # f.write('#define WAVE_DATA {%s' %quant_wav_tensor)
    f.write('#define WAVE_DATA {')
    # for i in quant_wav_tensor:

    # for i in range(16000)
    #  f.write('%s, ' %quant_wav_tensor[i])
    count = 0
    while count < 16000:
        f.write('%s, ' % quant_wav_tensor[count])
        count += 1
    f.write('}')
print("done with loop")

files.download('wav_data.h')



Starting
0ba018fc_nohash_2
wave data tensor
[ 1.8310547e-04  1.8310547e-04  2.7465820e-04 ... -6.1035156e-05
 -3.0517578e-05 -3.6621094e-04]
Quantized data tensor
[  6   6   9 ...  -2  -1 -12]
16000
done with loop
